You have 10 examples of fruit. For each example, you have
Three features
Color (Brown or Red),
throns_Shape (Tapering  or Enlarging), and
Solitary (Yes or No)
Label
Edible (1 indicating yes or 0 indicating poisonous)

For ease of implementation, we have one-hot encoded the features (turned them into 0 or 1 valued features)

In [1]:
import numpy as np
x_train = np.array([[1,1,1],[1,0,1],[1,0,0],[1,0,0],[1,1,1],[0,1,1],[0,0,0],[1,0,1],[0,1,0],[1,0,0]])
y_train = np.array([1,1,0,0,1,0,0,1,1,0])



In [2]:
def compute_entropy(y):
  entropy=0
  if len(y)!=0:
    p1=len(y[y==1])/len(y)
    if p1!=0 and p1!=1:
        entropy = -p1 * np.log2(p1) - (1 - p1) * np.log2(1 - p1)
    else:
             entropy = 0
  return entropy


In [4]:
z=compute_entropy(y_train)
print(z)

1.0


In [24]:
def split_dataset(x,node_indices,feature):

  node_indices=[0,1,2,3,4,5,6,7,8,9]

  left_indices=[]
  right_indices=[]
  for i in node_indices:

    if np.any(x[i][feature]==1):
         left_indices.append(i)
    else:
            right_indices.append(i)
  return left_indices,right_indices

In [5]:
node_indices=[0,1,2,3,4,5,6,7,8,9]
feature=0
left_indices,right_indices=split_dataset(x_train,node_indices,feature)
print('left=',left_indices)
print ('right=',right_indices)
feature=1
left_indices,right_indices=split_dataset(x_train,node_indices,feature)
print('left=',left_indices)
print ('right=,',right_indices)
feature=2
left_indices,right_indices=split_dataset(x_train,node_indices,feature)
print('left=',left_indices)
print ('right=',right_indices)





left= [0, 1, 2, 3, 4, 7, 9]
right= [5, 6, 8]
left= [0, 4, 5, 8]
right=, [1, 2, 3, 6, 7, 9]
left= [0, 1, 4, 5, 7]
right= [2, 3, 6, 8, 9]


In [29]:
def compute_infogain(x,y,feature,node_indices):
    left_indices,right_indices=split_dataset(x_train,node_indices,feature)
    X_node, y_node = x[node_indices], y[node_indices]
    X_left, y_left = x[left_indices], y[left_indices]
    X_right, y_right = x[right_indices], y[right_indices]
    information_gain=0
    node_entropy=compute_entropy(y_node)
    left_entropy=compute_entropy(y_left)
    right_entropy = compute_entropy(y_right)

    w_left = len(X_left) / len(X_node)
    w_right = len(X_right) / len(X_node)

    weighted_entropy = w_left * left_entropy + w_right * right_entropy
    information_gain = node_entropy - weighted_entropy
    return information_gain



In [7]:
root_indices=[0,1,2,3,4,5,6,7,8,9]
feature=0
compute_infogain(x_train,y_train,feature,root_indices)
print('Information Gain from splitting the root on brown cap:',  compute_infogain(x_train,y_train,feature,root_indices))
feature=1
compute_infogain(x_train,y_train,feature,root_indices)
print('Information Gain from splitting the root on tapering stalk shape:',compute_infogain(x_train,y_train,feature,root_indices ))
feature=2
print('Information Gain from splitting the root on solitary:',compute_infogain(x_train,y_train,feature,root_indices ))



Information Gain from splitting the root on brown cap: 0.034851554559677034
Information Gain from splitting the root on tapering stalk shape: 0.12451124978365313
Information Gain from splitting the root on solitary: 0.2780719051126377


In [8]:
def get_best_split(x, y, node_indices):

    num_features = x.shape[1]


    best_feature = None
    max_info_gain=0
    for feature in range(num_features):
        info_gain = compute_infogain(x, y, feature,node_indices)
        if info_gain > max_info_gain:
            max_info_gain = info_gain
            best_feature = feature
    return best_feature

In [9]:
root_indices= [0,1,2,3,4,5,6,7,8,9]
best_feature = get_best_split(x_train, y_train, root_indices)
print("Best feature to split on: ", best_feature)



Best feature to split on:  2


In [28]:
def build_tree_recursive(x, y, node_indices, branch_name, max_depth, current_depth, tree):


    if current_depth == max_depth:
        tree.append((branch_name, node_indices))
        return


    best_feature = get_best_split(x, y, node_indices)


    left_indices, right_indices = split_dataset(x, node_indices, best_feature)
    tree.append((branch_name, best_feature, left_indices, right_indices))


    build_tree_recursive(x, y, left_indices, "Left", max_depth, current_depth + 1, tree)
    build_tree_recursive(x, y, right_indices, "Right", max_depth, current_depth + 1, tree)


tree = []

build_tree_recursive(x_train, y_train, root_indices, "Root", 2, 0, tree)


for node in tree:
    branch_name = node[0]
    if branch_name == "Root":
        print("%s: Split on feature %s" % (branch_name, node[1]))
    else:
        if len(node) == 4 and node[1] is not None:
            print("%s Depth %d, %s: Split on feature %d" % (branch_name, node[1], node[2], node[3]))
        else:
            print("%s Depth None, %s: Leaf node with indices" % (branch_name, node[1]))




Root: Split on feature 2
Left Depth None, None: Leaf node with indices
Left Depth None, [0, 1, 2, 3, 4, 5, 7, 8, 9]: Leaf node with indices
Right Depth None, [6]: Leaf node with indices
Right Depth None, None: Leaf node with indices
Left Depth None, [0, 1, 2, 3, 4, 5, 7, 8, 9]: Leaf node with indices
Right Depth None, [6]: Leaf node with indices


In [ ]:
print(x_train.shape)

(10, 3)
